# api

> Main entrypoint api for beetroot

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *
from IPython.display import display, Markdown
import json
import os
import tempfile
from pathlib import Path

In [ ]:
#| hide
#| export
import io
from typing import Dict, Iterable, Tuple

In [ ]:
# | export
from beetroot.source import emit_markdown_source, emit_python_source
from beetroot.outputs import (
    Completion,
    emit_display_data_output,
    emit_execute_result_output,
    emit_stream_output,
)

In [ ]:
# | export
def export_notebook(nb_json: Dict) -> Tuple[str, Iterable[Completion]]:
    stream = io.StringIO()
    completions = []
    for cell in nb_json['cells']:
        if cell['cell_type'] == 'markdown':
            emit_markdown_source(cell['source'], stream)
            stream.write('\n')
        elif cell['cell_type'] == 'code':
            should_show_output = emit_python_source(cell['source'], stream)
            stream.write('\n')
            
            if not should_show_output:
                continue
            
            for output in cell['outputs']:
                output_type = output['output_type']
                if output_type == 'stream':
                    emit_stream_output(output, stream)
                elif output_type == 'display_data':
                    completion = emit_display_data_output(output, stream)
                    if completion:
                        completions.append(completion)
                elif output_type == 'execute_result':
                    completion = emit_execute_result_output(output, stream)
                    if completion:
                        completions.append(completion)
                stream.write('\n')
    
    stream.seek(0)
    return stream.read(), completions
    

In [ ]:
# Form the path to the testcase notebook
nbpath = os.path.dirname(os.path.realpath('__file__')) # Get path to this notebook
test_nb_path = Path(nbpath) / '../testcase_notebooks' / 'testcase_notebook.ipynb'

# Load the testcase notebook as JSON
nb_json = json.loads(test_nb_path.read_text())

# Call export, get the text and the completions
md_text, completions = export_notebook(nb_json)

# Create a temporary directory we'll use for running the completions
with tempfile.TemporaryDirectory() as tmpdir:
    tmp_path = Path(tmpdir)
    for completion in completions:
        completion(tmp_path)
        
    filename = None 
    with open(tmp_path / 'markdown.md', 'w') as md_file:
        md_file.write(md_text)
        filename = md_file.name 

    # Images won't render because their paths are relative to the markdown file,
    # not this notebook. That's unfortunate but I've decided to live with it. 
    # The solution would be to temporarily copy images from the output directory
    # to a local images/ directory here. But that seems brittle so I'm not doing
    # it for now. 
    display(Markdown(filename=filename))

```python
name = 'world'
print(f"hello, {name}")
```


```
hello, world
```

# This is Markdown
This is a [link](https://www.google.com).
This is:

* a 
* bulleted
* list

Yup.

```python
# Multiline stream output
print("hello, world")
print("a second line")
print("a third line")
```


```
hello, world
a second line
a third line
```

```python
# Single line stream output with no trailing newline
import sys
sys.stdout.write("hello")
```


```
hello
```

```
5
```

```python
# Multiline stream output with no trailing newline
print("hello with a newline")
sys.stdout.write("hello")
```


```
hello with a newline
hello
```

```
5
```


```
This output should show but the source code shouldn't
```

```python
# The source code for this cell will show but its output won't 

print("This output should not show")
```


The cell below is intentionally left blank to test handling blank cells - do not remove it

```python

```


The above cell was intentionally left blank to test handling blank cells - do not remove it

```python
# Test latex output
from IPython.display import Latex

Latex('$\\displaystyle w_{11} x_{11} + w_{12} x_{12} + w_{21} x_{21} + w_{22} x_{22}$')
```


$\displaystyle w_{11} x_{11} + w_{12} x_{12} + w_{21} x_{21} + w_{22} x_{22}$

```python
# Test inline image
%matplotlib inline 
from matplotlib import pyplot as plt
from math import cos, radians, sin 

fig = plt.figure(figsize=(2, 1))
ax = fig.add_subplot(1, 1, 1)
ax.set(
    title="sin(x) and cos(x)"
)
xs = range(0,360)
ax.plot(xs, [sin(radians(x)) for x in xs], label="sin(x)")
ax.plot(xs, [cos(radians(x)) for x in xs], label="cos(x)")
_ = ax.legend()
```


![](images/0872628e-eae0-4e00-878a-a7b966c1636c.png)
```python

```




In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()